In [ ]:
!pip install mlflow

In [127]:
import shutil
import os
import glob
import pandas as pd
import mlflow
#shutil.make_archive("mlruns", 'zip', "mlruns/")

import plotly.express as px
import matplotlib.pyplot as plt

In [132]:
out = []
out2 = {}
for p in glob.glob("mlruns/1/*/metrics/*"):
    
    run_id = p.split("/")[2]
    metric_type = p.split("/")[-1]
    
    df = pd.read_csv(p, sep = " ", names=["timestep", "value", "step"])
    df["run_id"] = run_id
    df["metric_type"] = metric_type
    
    out.append(df)
    
    out2[run_id] = mlflow.get_run(run_id)._data.params

out = pd.concat(out, axis=0).set_index(["step","run_id", "metric_type"])
out = out["value"].unstack()

out2 = pd.DataFrame(out2).T
out = out.reset_index().join(out2, on = "run_id")

#Filtering based on the best accuracy_test for each run.
out["test_score"] = out.sort_values("accuracy_test", ascending = False).groupby("run_id").cumcount()
out = out.query("test_score == 0")

In [133]:
out.sort_values("mae_test")

,step,run_id,accuracy_test,accuracy_train,loss_test,loss_train,mae_test,mae_train,optimizer_fn,n_layers,lr_scheduler,lr,model_name,cb_matrix,loss_fn,weight_decay,test_score
853,19,e36b544033ae43d89c6e2b627962ce19,0.694444,0.931637,10.174322,1.589243,0.351449,0.070786,SGD,32,True,0.001,VGG,"tensor([[ 5., 4., 7., 10., 13.],\n [ ...",CSCELoss(),0.0005,0
412,10,2e68d07e41da4c328b8a70f4c2cea713,0.685990,0.789373,10.869173,7.036013,0.354469,0.231222,SGD,32,True,0.001,VGG,"tensor([[10., 4., 7., 10., 13.],\n [ ...",CSCELoss(),0.0005,0
370,9,3dc6b838a67942e1a2876f783e644c5a,0.685990,0.742991,3.626537,2.796815,0.359300,0.288681,SGD,32,True,0.001,VGG,"tensor([[ 1., 4., 7., 10., 13.],\n [ ...",CSCELoss(),0.0005,0
516,12,829c255bbf4c4a39a922332a6885037f,0.685990,0.848737,12.621995,5.357849,0.364130,0.163724,SGD,32,True,0.0005,VGG,"tensor([[10., 4., 7., 10., 13.],\n [ ...",CSCELoss(),0.0005,0
356,8,c931269fd52b4987972ec1d5f324cf29,0.694444,0.739702,9.467808,7.684398,0.365942,0.296989,SGD,32,True,0.001,VGG,"tensor([[10., 3., 5., 7., 9.],\n [ ...",CSCELoss(),0.0005,0
562,13,86e2af01ce5b4dea995c51872b58d83f,0.693237,0.843718,10.777530,4.628115,0.367754,0.172378,SGD,32,True,0.0005,VGG,"tensor([[10., 2., 3., 4., 5.],\n [ ...",CSCELoss(),0.0005,0
662,15,abafee70d39648e8852327ee0be00b59,0.686594,0.897196,13.138713,3.279079,0.370169,0.110419,SGD,32,True,0.001,VGG,"tensor([[10., 2., 3., 4., 5.],\n [ ...",CSCELoss(),0.0005,0
281,7,5308688dc65147a5803af71ae0931ad4,0.692633,0.741779,6.128823,4.779095,0.370773,0.293181,SGD,32,True,0.001,VGG,"tensor([[5., 3., 5., 7., 9.],\n [3., 5....",CSCELoss(),0.0005,0
819,19,3a56c022667c4dd9b3e286f943dcf19e,0.682367,0.904638,7.596590,1.936503,0.372585,0.103496,SGD,32,True,0.0005,VGG,"tensor([[5., 3., 5., 7., 9.],\n [3., 5....",CSCELoss(),0.0005,0
658,15,a1ae903bcf0e4db3a176784dbbc87667,0.673913,0.835410,6.032678,2.757250,0.373188,0.182762,SGD,32,True,0.0005,VGG,"tensor([[5., 2., 3., 4., 5.],\n [2., 5....",CSCELoss(),0.0005,0


In [135]:
a = px.scatter(out, x = "accuracy_test", y = "mae_test", hover_data = ["lr", "cb_matrix"])

with open("ordinal_performances.html","w") as f:
    f.write(a.to_html())

In [136]:
out.to_csv("ordinal_performances.csv")